In [1]:
cd ..

/Users/adobles/projects/snap/relbench-user-study/amazon


## Feature Set

- `weeks_since_first_review`
- `num_reviews`
- `sum_review_ratings`
- `avg_review_length`
- `last_review_weeks_ago`
- `last_review_summary_text`: Embedding
- `last_reviewed_product_title`: Embedding
- `last_reviewed_product_category`
- `last_review_is_verified`
- `avg_review_rating`
- `pct_verified_reviews`
- `std_review_rating`
- `min_review_rating`
- `max_review_rating`
- `avg_reviewed_product_rating`
- `sum_reviewed_product_rating`
- `std_reviewed_product_rating`
- `min_reviewed_product_rating`
- `max_reviewed_product_rating`
- `avg_reviewed_product_price`
- `sum_reviewed_product_price`
- `std_reviewed_product_price`
- `min_reviewed_product_price`
- `max_reviewed_product_price`
- `reviewed_product_modal_category`
- `user_bias`: average over products the user reviewed of user's_review - avg_product_review / std_product review
- trend stuff: avg value in last 6 months - avg value in previous 6 month window
    - `num_reviews_trend`
    - `avg_rating_trend`
    - `avg_price_trend`
    - `avg_user_bias_trend`


*Took 1 hr*


In [2]:
import duckdb
import numpy as np
from relbench.datasets import get_dataset
from torch_frame import TaskType, stype
from torch_frame.gbdt import LightGBM, XGBoost
from torch_frame.data import Dataset
from torch_frame.typing import Metric
from torch_frame.utils import infer_df_stype

import utils

conn = duckdb.connect('amazon.db')
%load_ext sql
%sql conn --alias duckdb
%config SqlMagic.displaycon=False

### Scrapwork

In [8]:
%%sql
with age_proxy as (
    select
        churn_train.customer_id,
        churn_train.timestamp,
        churn_train.churn,
        max(date_diff('month', review.review_time, churn_train.timestamp)) as age_months
    from churn_train
    left join review
        on churn_train.customer_id = review.customer_id
        and review.review_time < churn_train.timestamp
    group by all
)
select
    churn,
    avg(age_months) as avg_age_at_eval,
    stddev(age_months) as std_age_at_eval,
    min(age_months) as min_age_at_eval,
    median(age_months) as median_age_at_eval,
    max(age_months) as max_age_at_eval
from age_proxy
group by churn

churn,avg_age_at_eval,std_age_at_eval,min_age_at_eval,median_age_at_eval,max_age_at_eval
0,31.017596107409734,31.92294067018162,0,19.0,187
1,28.006093508626318,28.51492883061461,0,19.0,180


In [10]:
%%sql
with reviews_at_eval as (
    select
        churn_train.customer_id,
        churn_train.timestamp,
        churn_train.churn,
        min(date_diff('weeks', review.review_time, churn_train.timestamp)) as last_review_weeks_ago,
        count(*) as num_reviews_at_eval,
        avg(date_diff('weeks', review.review_time, churn_train.timestamp)) as avg_review_age_at_eval
    from churn_train
    left join review
        on churn_train.customer_id = review.customer_id
        and review.review_time < churn_train.timestamp
    group by all
)

select
    churn,
    avg(last_review_weeks_ago) as avg_last_review_weeks_ago,
    median(last_review_weeks_ago) as median_last_review_weeks_ago,
    avg(num_reviews_at_eval) as avg_num_reviews_at_eval,
    avg(avg_review_age_at_eval) as avg_avg_review_age_at_eval
from reviews_at_eval
group by churn

churn,avg_last_review_weeks_ago,median_last_review_weeks_ago,avg_num_reviews_at_eval,avg_avg_review_age_at_eval
0,33.74813964719506,25.0,8.672029157572206,80.89926389942515
1,45.43446155300289,42.0,4.1545420746065185,83.89071972965345


In [12]:
%%sql
select
    verified,
    count(*),
    count(*) / sum(count(*)) over () as pct
from review
group by verified

verified,count_star(),pct
False,4611426,0.33616484683496245
True,9106326,0.6638351531650375


In [15]:
%%sql
select
    category[-1]
from product
limit 10

category[-1]
Dramas & Plays
Literature & Fiction
"Mystery, Thriller & Suspense"
Humor & Entertainment
Bible Study & Reference
Literature & Fiction
Bible Study & Reference
Animals
Literature & Fiction
Literature & Fiction


## Tuning

In [3]:
with open('churn/feats.sql', 'r') as f:
    # run once with train_labels and once with val_labels
    template = f.read()

# create train, val and test features
# takes 1 - 5 mins
for s in ['train', 'val', 'test']:
    print(f'Creating {s} table')
    query = utils.render_jinja_sql(template, dict(set=s))
    conn.sql(query)
    print(f'{s} table created')

Creating train table
train table created
Creating val table
val table created
Creating test table
test table created


In [4]:
train_df = conn.sql('select * from churn_train_feats').df()
val_df = conn.sql('select * from churn_val_feats').df()

In [5]:
col_to_stype = infer_df_stype(train_df)

In [6]:
col_to_stype

{'customer_id': <stype.numerical: 'numerical'>,
 'timestamp': <stype.timestamp: 'timestamp'>,
 'churn': <stype.categorical: 'categorical'>,
 'num_reviews': <stype.numerical: 'numerical'>,
 'sum_review_ratings': <stype.numerical: 'numerical'>,
 'avg_review_length': <stype.numerical: 'numerical'>,
 'last_review_weeks_ago': <stype.numerical: 'numerical'>,
 'last_review_summary_text': <stype.text_embedded: 'text_embedded'>,
 'last_reviewed_product_title': <stype.text_embedded: 'text_embedded'>,
 'last_reviewed_product_category': <stype.text_embedded: 'text_embedded'>,
 'last_review_is_verified': <stype.categorical: 'categorical'>,
 'avg_review_rating': <stype.numerical: 'numerical'>,
 'pct_verified_reviews': <stype.numerical: 'numerical'>,
 'std_review_rating': <stype.numerical: 'numerical'>,
 'min_review_rating': <stype.categorical: 'categorical'>,
 'max_review_rating': <stype.categorical: 'categorical'>,
 'avg_reviewed_product_rating': <stype.numerical: 'numerical'>,
 'sum_reviewed_produ

In [11]:
DROP_COLS = [
    # drop identifier cols
    'customer_id',
    'timestamp',
    # drop text embeds to save time
    'last_review_summary_text',
    'last_reviewed_product_title',
]
for c in DROP_COLS:
    del col_to_stype[c]
# Correct certain columns
col_to_stype['last_reviewed_product_category'] = stype.categorical
col_to_stype['min_review_rating'] = stype.numerical
col_to_stype['max_review_rating'] = stype.numerical
col_to_stype['reviewed_product_modal_category'] = stype.categorical

In [12]:
train_dset = Dataset(
    train_df.drop(DROP_COLS, axis=1),
    col_to_stype=col_to_stype,
    target_col='churn'
).materialize()
val_tf = train_dset.convert_to_tensor_frame(val_df.drop(DROP_COLS, axis=1))
tune_metric = Metric.ROCAUC
print(train_dset.tensor_frame.num_cols, train_dset.tensor_frame.num_rows)

26 724664


In [13]:
gbdt = LightGBM(TaskType.BINARY_CLASSIFICATION, num_classes=2, metric=tune_metric)
gbdt.tune(tf_train=train_dset.tensor_frame, tf_val=val_tf, num_trials=10)

/Users/adobles/opt/anaconda3/envs/relbench/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-05-16 22:05:41,208] A new study created in memory with name: no-name-0f392e88-49dd-4887-bb3b-5ade30853bd7
[I 2024-05-16 22:06:01,087] Trial 0 finished with value: 0.6082582360049349 and parameters: {'booster': 'gblinear', 'use_lambda': False, 'use_alpha': True, 'alpha': 1.7045902607297673e-07}. Best is trial 0 with value: 0.6082582360049349.
[I 2024-05-16 22:06:10,167] Trial 1 finished with value: 0.5505843230327595 and parameters: {'booster': 'gblinear', 'use_lambda': False, 'use_alpha': True, 'alpha': 8.655286144974532}. Best is trial 0 with value: 0.6082582360049349.
[I 2024-05-16 22:06:40,678] Trial 2 finished with value: 0.6540593170841662 and parameters: {'booster': 'gbtree', 'use_lambda': True, 'lam

KeyboardInterrupt: 